In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d akkulkarni/training-handwritten-marathi-character-fusion

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip training-handwritten-marathi-character-fusion.zip

unzip:  cannot find or open training-handwritten-marathi-character-fusion.zip, training-handwritten-marathi-character-fusion.zip.zip or training-handwritten-marathi-character-fusion.zip.ZIP.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=None,
)

In [ ]:
path = '/content/Data_set'
size = (128, 128)


train = datagen.flow_from_directory(
    path,
    target_size=size,
    batch_size=64,
    class_mode='categorical',
    subset="training",
    seed=123,
)

val = datagen.flow_from_directory(
    path,
    target_size=size,
    batch_size=64,
    class_mode='categorical',
    subset="validation",
    seed=123,
)

FileNotFoundError: ignored

In [ ]:
train.class_indices

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from keras.models import Sequential

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3),
)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense((len(train.class_indices)), activation='softmax'))


model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)


model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    'model.h5',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    verbose=1)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.3,
    patience=3,
    verbose=1)


history = model.fit(train,
          validation_data=val,
          epochs=30,
          verbose=1,
          callbacks=[checkpoint, early_stopping, reduce_lr])

In [ ]:
with open('model.json', 'w') as json_file:
    json_file.write(model.to_json())

model.save_weights('model.h5')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.show()

In [ ]:
accu = model.evaluate(val)[1]
print('Final Test Acccuracy = {:.3f}'.format(accu*100))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(val)
y_pred_int = y_pred.argmax(axis=1)

plt.figure(figsize = (15,11))
sns.heatmap(confusion_matrix(val.classes, y_pred_int), annot = True,fmt="d",cmap = "Blues")
plt.show()